<a href="https://colab.research.google.com/github/YCCS-Summer-2023-DDNMA/project/blob/80-practice-with-neural-networks-mk/Michael_Kupferstein/nn_from_scratch/nnScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP training on MNIST

In [ ]:
# todo: init MPL and add the predict
# todo: add data loading in PyTorch
# todo: add the training loop, loss fn

# Visualizations

In [ ]:
# todo: visualize the MPL weight
# todo: visulaize emedding using t-SNE
# todo: dead neurons

# Parallelization